In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
import os
from csv import writer


In [2]:
def scrape_url(url):
    response=requests.get(url)        #get requests
    response.status_code                  #get status code
    soup = BeautifulSoup(response.content, 'html.parser')
    results=soup.find_all('div',{'class':'vehicle-card'})
    scraped_data=[]
    for result in results:
        try:
            vehicle=result.find("h2").get_text().split()
            year=vehicle[0]
            vehicle_name=" ".join(vehicle[1:])
            miles=result.find('div',{'class':'mileage'}).get_text().strip("mi.")
            price=result.find('span',{'class':'primary-price'}).get_text().strip("$")
            vehicle_card_visited_tracking_link = result.find('a' ,{'class':'vehicle-card-visited-tracking-link'})
            d=vehicle_card_visited_tracking_link.attrs

            time.sleep(1)
            response=requests.get("https://www.cars.com/"+d['href']) 
            soup = BeautifulSoup(response.content, 'html.parser')
            per_vehicle = soup.find('dl',{'class':"fancy-description-list"})

            transmission=per_vehicle.find("dt",text="Transmission").findNext("dd").string.strip()
            fuel_type=per_vehicle.find("dt",text="Fuel type").findNext("dd").string
            drivetrain=per_vehicle.find("dt",text="Drivetrain").findNext("dd").string
            exterior_color=per_vehicle.find("dt",text="Exterior color").findNext("dd").string
            vin=per_vehicle.find("dt",text="VIN").findNext("dd").string
            mpg=per_vehicle.find("dt",text="MPG").findNext("dd").string
            engine=per_vehicle.find("dt",text="Engine").findNext("dd").string
            scraped_data.append([vehicle_name,year,miles,price,transmission,fuel_type,drivetrain,exterior_color,vin,mpg,engine])
        except:
            print(f"Scrape failed for {result}")
    return scraped_data

In [3]:
def scrape_only_price(url):
    response=requests.get(url)        #get requests
    response.status_code                  #get status code
    soup = BeautifulSoup(response.content, 'html.parser')
    results=soup.find_all('div',{'class':'vehicle-card'})
    scraped_data=[]
    for result in results:
        try:
            vehicle=result.find("h2").get_text().split()
            year=vehicle[0]
            vehicle_name=" ".join(vehicle[1:])
            price=result.find('span',{'class':'primary-price'}).get_text().strip("$")
            

            time.sleep(1)
            
            scraped_data.append([vehicle_name,year,price])
        except:
            print(f"Scrape failed for {result}")
    return scraped_data

In [4]:
def generate_and_scrape_pages_url(number_of_pages,page_size):
    
    
    #url="https://www.cars.com/shopping/results/?page={}&page_size={}&dealer_id=&keyword=&list_price_max=&list_price_min=&maximum_distance=30&mileage_max=&sort=best_match_desc&stock_type=used&year_max=&year_min=&zip=94544"
    
    #This is for used SUV's (2016 onwards) only which has 7000 entries for 50 miles radius from hayward
    #url="https://www.cars.com/shopping/results/?page={}&page_size={}&body_style_slugs[]=suv&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=50&mileage_max=&sort=list_price&stock_type=used&year_max=&year_min=2016&zip=94544"
    #csv_file= open('suv_data.csv', 'w', newline='')
    
    #This is for used sedans (2016 onwards) only which has around 4500 entries for 50 miles radius form hayward
#     url="https://www.cars.com/shopping/results/?page={}&page_size={}&body_style_slugs[]=sedan&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=50&mileage_max=&sort=best_match_desc&stock_type=used&year_max=&year_min=2016&zip=94544"
#     csv_file= open('sedan_data.csv', 'w', newline='')
    
    
    #This URL if for new sedan 50 miles radius of Hayward
    #url="https://www.cars.com/shopping/results/?page={}&page_size=&body_style_slugs[]=sedan&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=50&mileage_max=&sort=best_match_desc&stock_type=new&year_max=&year_min=2016&zip=94544
    #csv_file= open('cars.csv', 'w', newline='')
    
    url="https://www.cars.com/shopping/results/?page={}&page_size={}&body_style_slugs[]=suv&body_style_slugs[]=sedan&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=50&mileage_max=&sort=year_desc&stock_type=used&year_max=2015&year_min=2005&zip=94544"
    csv_file= open('suv_sedan_09_15.csv', 'w', newline='')
    
    w=writer(csv_file)
    w.writerow(['vehicle_name','year','miles','price','transmission','fuel_type','drivetrain','exterior_color','vin','mpg','engine'])
    rownum=1
    for i in range(1,number_of_pages):
        rendered_url=url.format(i,page_size)
        print(rendered_url)
        scraped_data=scrape_url(rendered_url)
        for row in scraped_data:
            print(f"writing: page_number {i+1} row_number {rownum} {row[8]}")
            w.writerow(row)
            rownum +=1
        
        csv_file.flush()
        os.fsync(csv_file)
         
    csv_file.close()

In [5]:
generate_and_scrape_pages_url(220,20)

https://www.cars.com/shopping/results/?page=1&page_size=20&body_style_slugs[]=suv&body_style_slugs[]=sedan&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=50&mileage_max=&sort=year_desc&stock_type=used&year_max=2015&year_min=2005&zip=94544
writing: page_number 2 row_number 1  WP1AB2A54FLB42409 
writing: page_number 2 row_number 2  WBS3C9C54FP804231 
writing: page_number 2 row_number 3  2FMTK4K90FBB35105 
writing: page_number 2 row_number 4  JTHBF1D20F5065756 
writing: page_number 2 row_number 5  WP1AB2A59FLB64809 
writing: page_number 2 row_number 6  WP1AB2A54FLB74437 
writing: page_number 2 row_number 7  2FMTK4K93FBB51041 
writing: page_number 2 row_number 8  WDDSJ4EBXFN209609 
writing: page_number 2 row_number 9  WDDHF5KB5FB122904 
writing: page_number 2 row_number 10  4JGDA5HB5FA486000 
writing: page_number 2 row_number 11  1C4RJFCM5FC767520 
writing: page_number 2 row_number 12  1HGCR2F81FA155320 
writing: page_number 2 row_number 13  JTHBF1D26F5077894

Scrape failed for <div class="vehicle-card" data-listing-id="3a9d4933-2dbc-4014-8b97-5d38cf6f14ec" data-tracking-id="16" data-tracking-type="srp-vehicle-card" id="3a9d4933-2dbc-4014-8b97-5d38cf6f14ec" phx-hook="VehicleCard" phx-target="20">
<a class="vehicle-card-visited-tracking-link" href="/vehicledetail/3a9d4933-2dbc-4014-8b97-5d38cf6f14ec/" rel="nofollow" tabindex="-1"></a>
<div class="vehicle-card-main js-gallery-click-card">
<div class="vehicle-card-photos js-gallery-click-gallery">
<div class="gallery-wrap">
<div aria-label="Photo 1 of 23 of 2015 Honda Civic SE" class="image-wrap" data-index="0">
<img alt="Photo 1 of 23 of 2015 Honda Civic SE" class="vehicle-image js-lazy-load styles-lazy-load" data-src="https://platform.cstatic-images.com/large/in/v2/f464ff1d-5120-5dfa-9f09-57910e478bae/c8c1f7c7-c7f2-4abf-85ce-45d18611d361/DyTznA17JC5lvcgcF160vzCPkYg.jpg" src="/images/placeholder_10x10.png"/>
</div>
<div aria-label="Photo 2 of 23 of 2015 Honda Civic SE" class="image-wrap" data-

writing: page_number 8 row_number 119  JTEBU5JR0F5267850 
writing: page_number 8 row_number 120  WBA5B1C50FD921677 
writing: page_number 8 row_number 121  1N4AL3AP2FC225316 
writing: page_number 8 row_number 122  5N1AT2MT3FC778744 
writing: page_number 8 row_number 123  JTHBE1D29F5022035 
writing: page_number 8 row_number 124  JTHBF1D22F5066875 
writing: page_number 8 row_number 125  2HGFB2F77FH538365 
writing: page_number 8 row_number 126  3GYFNCE39FS614217 
writing: page_number 8 row_number 127  WDCGG0EB1FG419165 
writing: page_number 8 row_number 128  SALGS2TF7FA239545 
writing: page_number 8 row_number 129  1GKS2CKJ5FR718310 
writing: page_number 8 row_number 130  JTHGL5EF3F5054242 
writing: page_number 8 row_number 131  19XFB2F5XFE251479 
writing: page_number 8 row_number 132  19XFB4F38FE200234 
writing: page_number 8 row_number 133  5UXKR0C57F0P11658 
writing: page_number 8 row_number 134  2GNALBEK2F6365420 
writing: page_number 8 row_number 135  JTEBU5JR4F5234687 
writing: page_

writing: page_number 14 row_number 238  JTEZU5JR0F5093759 
writing: page_number 14 row_number 239  WA1LGAFE3FD024144 
writing: page_number 14 row_number 240  1FAHP2F8XFG118652 
writing: page_number 14 row_number 241  4T1BF1FK1FU977280 
writing: page_number 14 row_number 242  KNMAT2MV9FP571645 
writing: page_number 14 row_number 243  WDDSJ5CB4FN204074 
writing: page_number 14 row_number 244  5TDYKRFH9FS074814 
writing: page_number 14 row_number 245  1G4GA5G33FF215592 
writing: page_number 14 row_number 246  WMWZC5C52FWP46598 
writing: page_number 14 row_number 247  WBA5A7C52FG142916 
writing: page_number 14 row_number 248  JN8AZ2NE6F9081704 
writing: page_number 14 row_number 249  SALVP2BGXFH037633 
writing: page_number 14 row_number 250  1VWAT7A37FC123344 
writing: page_number 14 row_number 251  1N4AL3APXFC112696 
writing: page_number 14 row_number 252  WBA5B1C54FD918782 
writing: page_number 14 row_number 253  WA1LGAFE4FD021821 
writing: page_number 14 row_number 254  5YFBURHEXFP27193

writing: page_number 17 row_number 295  JTMRFREV8FD118541 
writing: page_number 17 row_number 296  4T1BF1FK3FU031181 
writing: page_number 17 row_number 297  SALGV2TF4FA217638 
writing: page_number 17 row_number 298  3MZBM1U77FM200872 
writing: page_number 17 row_number 299  2G1165S39F9249296 
writing: page_number 17 row_number 300  2T2BK1BA3FC266105 
writing: page_number 17 row_number 301  JTEBU5JR0F5215151 
writing: page_number 17 row_number 302  1C4PJLCBXFW547681 
writing: page_number 17 row_number 303  1GYS4CKJ7FR302842 
writing: page_number 17 row_number 304  WBAVM1C54FV319479 
writing: page_number 17 row_number 305  1VWAT7A39FC036285 
writing: page_number 17 row_number 306  5FRYD4H49FB030600 
writing: page_number 17 row_number 307  5N1AA0ND4FN600508 
writing: page_number 17 row_number 308  3FA6P0HDXFR137474 
writing: page_number 17 row_number 309  1FADP3F27FL319111 
writing: page_number 17 row_number 310  5YFBURHE0FP328440 
writing: page_number 17 row_number 311  WDDSJ5CB5FN20198

...

writing: page_number 195 row_number 3760  ZAM57RTA9F1131789 
writing: page_number 195 row_number 3761  19XFB2F54FE065940 
writing: page_number 195 row_number 3762  3FA6P0HD1FR280409 
writing: page_number 195 row_number 3763  1C3CCCBG1FN543017 
writing: page_number 195 row_number 3764  3N1AB7AP0FL688096 
writing: page_number 195 row_number 3765  WDDUG7JB5FA125512 
writing: page_number 195 row_number 3766  3N1AB7AP2FL668352 
writing: page_number 195 row_number 3767  5NPE34AF7FH098785 
writing: page_number 195 row_number 3768  2T1BURHE6FC272097 
writing: page_number 195 row_number 3769  1HGCR2F86FA010676 
writing: page_number 195 row_number 3770  WAUFGAFC7FN009137 
writing: page_number 195 row_number 3771  WAUBFAFL2FN042127 
writing: page_number 195 row_number 3772  5YFBURHE8FP210166 
writing: page_number 195 row_number 3773  2T1BURHE2FC315103 
writing: page_number 195 row_number 3774  1FADP3FE5FL346835 
writing: page_number 195 row_number 3775  55SWF4KBXFU010283 
writing: page_number 195

writing: page_number 201 row_number 3880  ZAM57RTA9F1131789 
writing: page_number 201 row_number 3881  19XFB2F54FE065940 
writing: page_number 201 row_number 3882  3FA6P0HD1FR280409 
writing: page_number 201 row_number 3883  1C3CCCBG1FN543017 
writing: page_number 201 row_number 3884  3N1AB7AP0FL688096 
writing: page_number 201 row_number 3885  WDDUG7JB5FA125512 
writing: page_number 201 row_number 3886  3N1AB7AP2FL668352 
writing: page_number 201 row_number 3887  5NPE34AF7FH098785 
writing: page_number 201 row_number 3888  2T1BURHE6FC272097 
writing: page_number 201 row_number 3889  1HGCR2F86FA010676 
writing: page_number 201 row_number 3890  WAUFGAFC7FN009137 
writing: page_number 201 row_number 3891  WAUBFAFL2FN042127 
writing: page_number 201 row_number 3892  5YFBURHE8FP210166 
writing: page_number 201 row_number 3893  2T1BURHE2FC315103 
writing: page_number 201 row_number 3894  1FADP3FE5FL346835 
writing: page_number 201 row_number 3895  55SWF4KBXFU010283 
writing: page_number 201

writing: page_number 204 row_number 3940  ZAM57RTA9F1131789 
writing: page_number 204 row_number 3941  19XFB2F54FE065940 
writing: page_number 204 row_number 3942  3FA6P0HD1FR280409 
writing: page_number 204 row_number 3943  1C3CCCBG1FN543017 
writing: page_number 204 row_number 3944  3N1AB7AP0FL688096 
writing: page_number 204 row_number 3945  WDDUG7JB5FA125512 
writing: page_number 204 row_number 3946  3N1AB7AP2FL668352 
writing: page_number 204 row_number 3947  5NPE34AF7FH098785 
writing: page_number 204 row_number 3948  2T1BURHE6FC272097 
writing: page_number 204 row_number 3949  1HGCR2F86FA010676 
writing: page_number 204 row_number 3950  WAUBFAFL2FN042127 
writing: page_number 204 row_number 3951  5YFBURHE8FP210166 
writing: page_number 204 row_number 3952  2T1BURHE2FC315103 
writing: page_number 204 row_number 3953  1FADP3FE5FL346835 
writing: page_number 204 row_number 3954  55SWF4KBXFU010283 
writing: page_number 204 row_number 3955  3VW2K7AJ8FM256005 
writing: page_number 204

writing: page_number 210 row_number 4059  ZAM57RTA9F1131789 
writing: page_number 210 row_number 4060  19XFB2F54FE065940 
writing: page_number 210 row_number 4061  3FA6P0HD1FR280409 
writing: page_number 210 row_number 4062  1C3CCCBG1FN543017 
writing: page_number 210 row_number 4063  3N1AB7AP0FL688096 
writing: page_number 210 row_number 4064  WDDUG7JB5FA125512 
writing: page_number 210 row_number 4065  3N1AB7AP2FL668352 
writing: page_number 210 row_number 4066  5NPE34AF7FH098785 
writing: page_number 210 row_number 4067  2T1BURHE6FC272097 
writing: page_number 210 row_number 4068  1HGCR2F86FA010676 
writing: page_number 210 row_number 4069  WAUFGAFC7FN009137 
writing: page_number 210 row_number 4070  WAUBFAFL2FN042127 
writing: page_number 210 row_number 4071  5YFBURHE8FP210166 
writing: page_number 210 row_number 4072  2T1BURHE2FC315103 
writing: page_number 210 row_number 4073  1FADP3FE5FL346835 
writing: page_number 210 row_number 4074  55SWF4KBXFU010283 
writing: page_number 210

writing: page_number 216 row_number 4179  ZAM57RTA9F1131789 
writing: page_number 216 row_number 4180  19XFB2F54FE065940 
writing: page_number 216 row_number 4181  3FA6P0HD1FR280409 
writing: page_number 216 row_number 4182  1C3CCCBG1FN543017 
writing: page_number 216 row_number 4183  3N1AB7AP0FL688096 
writing: page_number 216 row_number 4184  WDDUG7JB5FA125512 
writing: page_number 216 row_number 4185  3N1AB7AP2FL668352 
writing: page_number 216 row_number 4186  5NPE34AF7FH098785 
writing: page_number 216 row_number 4187  2T1BURHE6FC272097 
writing: page_number 216 row_number 4188  1HGCR2F86FA010676 
writing: page_number 216 row_number 4189  WAUFGAFC7FN009137 
writing: page_number 216 row_number 4190  WAUBFAFL2FN042127 
writing: page_number 216 row_number 4191  5YFBURHE8FP210166 
writing: page_number 216 row_number 4192  2T1BURHE2FC315103 
writing: page_number 216 row_number 4193  1FADP3FE5FL346835 
writing: page_number 216 row_number 4194  55SWF4KBXFU010283 
writing: page_number 216